<a href="https://colab.research.google.com/github/youngmoo/ECES-435/blob/main/Class5-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-435: Class 5.1**

**Announcements**
* This wext week's lab (No. 5) is the Midterm Lab (20% of final grade)
  * It's a simplified version of the final project (audio codec).
  * We'll try to release it tomorrow
  * Whenever it's released, you'll have a full week to work on it.




# Optional stuff

## Interactive Matplotlib

Install `ipympl` for  Matplotlib


In [ ]:
#!pip install ipympl   # Also installs a more recent version of matplotlib (v3.5.3)

Enable interactive Matplotlib figures

In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()
# %matplotlib widget

## My plot style defaults

In [ ]:
from matplotlib import rc

rc('figure', figsize=(12,4))
rc('figure', facecolor='#aaaaaa')     # Better figure background for dark mode

rc('font', family='Liberation Serif') # Nicer font
rc('font', size=20)                   # Larger font size for labels

# Setup
As always, start by importing the "usual" modules we'll be using...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import soundfile as sf
from matplotlib import animation, rc
from scipy import signal
import numpy.fft as fft     # New addition!

rc('animation', html='jshtml')

path = '/content/drive/My Drive/eces435-work/class5.1/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'jv24601'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Helper functions

## `myPlot()`
A quick time-domain signal plot function with my default figure settings and a time x-axis (in seconds).
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `N=#` Number of samples to plot (default: length of signal)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Plot format string (default: none)
  * *New* `x_lim=#` or `x_lim=[x1,x2]` Specify the x-axis limit(s) of the plot
  * *New* `y_lim=#` or `y_lim=[y1,y2]` Specificy the y-axis limit(s) of the plot

In [ ]:
def myPlot(sig, N=0, fs=44100, fig_size=(16,4), x_ax=True, y_ax=True, lw=1, fmt='', x_lim=0, y_lim=0):
  if N==0:
    N = len(sig)

  fig = plt.figure(figsize=fig_size)
  t = np.arange(N)/fs

  plt.plot( t[:N], sig[:N], fmt, linewidth=lw)

  plt.xlabel('Time (sec)')
  ax = plt.gca()    # gca(): "Get current axis", the graph object that's currently plotted
  
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = N/fs   # End of input signal
    else:
      x_lim2 = x_lim
    plt.xlim(0, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim != 0:
      plt.ylim(-y_lim, y_lim)
  else:
    plt.ylim(y_lim)

  fig.tight_layout()
  #plt.ion()
  plt.close()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

## `myPlotFFT()`
Plot the magnitude frequency response (in dB FS) of a signal with my default figure settings and a frequency x-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `n_fft=#` The size of FFT to use (default: length of input signal)
  * `n_win=#` The length of window to use (default: length of input)
  * `win='hann'` The type of window to use (default: `hann`, or `rect`)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `x_lim=# or (#,#)` Frequency axis limits (max or range, in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Change plot formatting (default: none)

In [ ]:
def myPlotFFT(sig, fs=44100, n_fft=0, n_win=0, win='hann', neg_f=False, x_lim=0, y_lim=0, fig_size=(16,6),x_ax=True, y_ax=True, lw=1, fmt=''):
  if n_fft==0:
    n_fft = len(sig)
  if n_win==0:
    n_win = len(sig)

  if win=='hann':  
    win = np.hanning(n_win)
    win_scale = 2
  else:
    win = np.ones(n_win)
    win_scale = 1

  S = np.fft.fft(sig * win, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  if neg_f:
    f = f - (fs/2)
    S = np.fft.fftshift(S)

  S_mag = 2*win_scale*np.abs(S) / n_win     # Frequency magnitude, normalized by length
                                            #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                                            #    x2 for Hann because window has 0.5 average

  S_mag += 1e-15                  # Add a small offset to avoid log(0) errors
  S_dBFS = 20*np.log10(S_mag)     # Freq. magnitude in dB full scale (dB FS):
                                  #    cos(w) -> 0 dBFS peak at w


  fig = plt.figure(figsize=fig_size)
  plt.plot(f, S_dBFS, fmt, linewidth=lw) 
  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = fs/2
    if neg_f:
      x_lim = -fs/2
    plt.xlim(x_lim, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim < 0:
      plt.ylim(y_lim, 0)
    elif y_lim > 0:
      plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB FS)')

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

## `mySpectrogram()`
A simple wrapper to compute and plot the spectrogram of a signal with my default figure settings, a time x-axis (in seconds) and a frequency y-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `win='window_name'` The type of analysis window to use (default: 'hann')
  * `n_win=#` The length of window to use per frame (default: 1024)
  * `n_fft=#` The size of FFT to use (default: 1024)
  * `x_lim=# or (#,#)` x-axis limit or range (in seconds)
  * `y_lim=# or (#,#)` y-axis limit or range (in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x-ax=True/False` Show x-axis (default: True)
  * `y-ax=True/False` Show y-axis (default: True)

In [ ]:
def mySpectrogram(sig, fs=44100, win='hann', n_win=1024, olap=512, n_fft=1024, x_lim=0, y_lim=0, fig_size=(12,6), x_ax=True, y_ax=True):
  f1, t1, Sxx = signal.stft(sig, fs, window=win, nperseg=n_win, noverlap=olap, nfft=n_fft)

  fig = plt.figure(figsize=fig_size)

  S_mag = 4*np.abs(Sxx) + 1e-15
  S_dBFS = 20*np.log10(S_mag)
  
  plt.pcolormesh(t1, f1, S_dBFS)
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim = len(sig) / fs
    plt.xlim(0, x_lim)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim == 0:
      y_lim = fs/2
    plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # plt.ion()
  
  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

# Bitrate

What's the bitrate for these various quantizations?
* $b_r = f_s \cdot B \cdot C$ in bits / second
  * $f_s$ is the sampling rate
  * $B$ is bits per sample
  * $C$ is the number of channels

In [ ]:
n_bits = 16
fs = 44100

br44_16 = fs * n_bits
print(br44_16)

# Load today's audio sample

* 44.1 kHz sampling rate

In [ ]:
x, fs44 = sf.read(path + 'BigTime.wav')
ipd.Audio(x, rate=fs44)

In [ ]:
myPlot(x)

# Our filter function, `smoothie()`

Inputs:
* `x` signal to be filtered
* `h` filter function

Output:
* `s` filtered signal

What it does:
* Implement the following operation:
  * $y[n] = \sum_{m=n}^{n+L} x[m] h[m-n]$

In [ ]:
def smoothie(x, h):
  N = len(x)
  L = len(h)

  y = np.zeros(N)                   # Create an output array of zeros
  for n in range(N - L):
    y[n] = np.sum( x[n:n+L] * h )

  return y

## Create a low-pass filter

In [ ]:
L = 17  # Number of samples to average
N = 50
h = np.zeros(N)
h[:L] = np.hanning(L)

plt.stem(h, basefmt=' ')

In [ ]:
y = smoothie(x, h)

In [ ]:
myPlot(x[:1500])

In [ ]:
myPlot(y[:1500])

In [ ]:
ipd.Audio(y, rate=fs44)

## What's another way of describing our equation?
* $y[n] = \sum_{m=n}^{n+L}x[m] h[m-n]$ ?
* Why do we call the filter function $h[n]$?

In [ ]:
y_c = np.convolve(x, h)
myPlot(y_c[:1500])

# What's going on in the frequency domain?

In [ ]:
myPlotFFT(x, win='rect')

In [ ]:
myPlotFFT(h, n_fft=2048, win='rect', y_lim=-100)

Plot the output, $X[k]H[k]$

In [ ]:
fig = 

Save it when you're done

In [ ]:
fig.savefig(path + username + '-X[k]H[k].png')

## What's up with the sidelobes?

In [ ]:
W = fft.fft(h, n=128)
f = np.arange(128)*fs44/128
plt.plot(f,np.real(W),f,np.imag(W))

In [ ]:
f2 = f-fs44/2
plt.plot(f2,fft.fftshift(np.real(W)),f2,fft.fftshift(np.imag(W)))

In [ ]:
plt.plot(f2, fft.fftshift(np.abs(W)))

## So, does multiplying in the frequency domain work?

In [ ]:
N = len(x)
X = np.fft.fft(x, n=N+L)
H = np.fft.fft(h, n=N+L)
Y = X * H

In [ ]:
Y_dB = 20*np.log10(np.abs(Y))
f = fs44 * np.arange(N+L)/ (N+L)
plt.plot(f, Y_dB)
plt.xlim(0,fs44/2)
plt.ylim(-100,100)

## New concept: The *inverse* Discrete Fourier Transform!

* $X[k] = \sum_{n=0}^{N} x[n] \cdot e^{-j \frac{2\pi \cdot k \cdot n}{N}}$

* $x[n] = \frac{1}{N} \sum_{k=0}^{N} X[k] \cdot e^{+j \frac{ 2\pi \cdot k \cdot n}{N}}$

In [ ]:
y_i = fft.ifft(Y)

In [ ]:
myPlot(y_i)

In [ ]:
myPlot(np.imag(y_i))

In [ ]:
ipd.Audio(y_i, rate=fs44)

## Longer audio sample

In [ ]:
x_20, fs44 = sf.read(path + 'BigTime20s.wav')
ipd.Audio(x_20, rate=fs44)

In [ ]:
N_20 = len(x_20)
X_20 = fft.fft(x_20)
H_20 = 
Y_20 = 

In [ ]:
y_20 = fft.ifft(Y_20)
fig2 = myPlot(y_20)

In [ ]:
fig2.savefig(path + username + 'y_20[n]')

In [ ]:
ipd.Audio(y_20, rate=fs44)

# Let's just multiply by the ideal LPF!
  * $H_{id} = 1$, $f \leq f_c$
  * $H_{id} = 0$, $f > f_c$

In [ ]:
print(N_20)

In [ ]:
2**20

In [ ]:
H_id = np.zeros(2**20)
f_c = 6000
n_c = f_c/fs44 * 2**20
print(n_c)
H_id[:int(n_c)] = 1
f = fs44 * np.arange(2**20) / 2**20
plt.plot(f, H_id)
plt.xlabel('Frequency (Hz)')

In [ ]:
H_id[-int(n_c):] = 1
plt.plot(f, H_id)
plt.xlabel('Frequency (Hz)')

In [ ]:
h_id = fft.ifft(H_id)
plt.plot(h_id)

In [ ]:
plt.plot(fft.fftshift(h_id))
plt.xlim(524000,524600)

In [ ]:
X_20_id = fft.fft(x_20, n=2**20)
Y_20_id = X_20_id * H_id

plt.plot(f, 20*np.log10(np.abs(Y_20_id + 1e-15)))
plt.xlim(0, fs44/2)
plt.xlabel('Frequency (Hz)')

In [ ]:
y_20_id = fft.ifft(Y_20_id)
myPlot(y_20_id)

In [ ]:
ipd.Audio(y_20_id, rate=fs44)

# Working with much longer signals
* Use the STFT (and Inverse STFT)

In [ ]:
mySpectrogram(np.real(y_20_id))

In [ ]:
f1, t1, S_20 = signal.stft(x_20, fs44, nperseg=256, noverlap=128, nfft=256)

S_mag = np.abs(S_20) + 1e-15
S_dB = 20*np.log10(S_mag)
  
plt.pcolormesh(t1, f1, S_dB)
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

In [ ]:
np.shape(S_20)

In [ ]:
S_20[10:, :] = 0

In [ ]:
S_mag = np.abs(S_20) + 1e-15
S_dB = 20*np.log10(S_mag)
  
plt.pcolormesh(t1, f1, S_dB)
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

In [ ]:
t2, y2 = signal.istft(S_20, nperseg=256, noverlap=128, nfft=256)
plt.plot(t2,y2)

In [ ]:
ipd.Audio(y2, rate=fs44)

In [ ]:
mySpectrogram(y2)